### Noah's Code

In [ ]:
import tensorflow as tf
def torch_autocorr(x):
    if len(x.shape) < 4:
        x = torch.unsqueeze(x, dim=0)

    x = x.detach().cpu().numpy()

    dim = x.shape[1]

    x = 2*x - 1
    x = tf.transpose(x, perm=[0, 3, 1, 2])
    x = tf.cast(x, tf.complex64)

    m = x
    M = np.array(tf.signal.fft2d(m))

    mag = tf.math.abs(M)
    mag = tf.cast(mag, tf.complex64)

    ang = tf.math.atan2(tf.math.imag(M), tf.math.real(M))
    ang = tf.cast(ang, tf.complex64)

    exp1 = tf.math.exp(tf.dtypes.complex(0., -1.)*ang)
    exp2 = tf.math.exp(tf.dtypes.complex(0., 1.)*ang)

    term1 = mag*exp1
    term2 = mag*exp2

    FFtmp = (term1*term2)/(32**2)

    autocorr = tf.signal.ifft2d(FFtmp)
    autocorr = tf.math.real(autocorr)

    autocorr = tf.transpose(autocorr, perm=[0, 2, 3, 1])

    rv = np.int32(np.floor(dim/2))

    # rv = tf.cast(tf.math.floor(dim/2), tf.int32)
    autocorr = tf.roll(autocorr, rv, 1)
    autocorr = tf.roll(autocorr, rv, 2)
    # autocorr = tf.signal.ifftshift(autocorr, axes=(1, 2))


    # Convert the TensorFlow tensor to a NumPy array
    np_array = autocorr.numpy()

    # Create a PyTorch tensor from the NumPy array
    torch_tensor_autocorr = torch.from_numpy(np_array)

    return torch_tensor_autocorr

### Torch

In [ ]:
import torch
import torch.nn as nn

def autocorr(x):
    """
    x is a 3 dimensional torch tensor. 
    """

    image_length = x.shape[1]

    x = 2*x - 1
    x = torch.permute(x, (0, 3, 1, 2))
    x_complex = x.type(torch.complex64)

    M = torch.fft.fft2(x_complex)
    mag_M_complex = torch.abs(M).type(torch.complex64)

    ang = torch.atan2(M.imag, M.real)
    ang = ang.type(torch.complex64)

    exp1 = torch.exp(torch.complex(torch.tensor([0], dtype=torch.float32), torch.tensor([-1], dtype=torch.float32))*ang)
    exp2 = torch.exp(torch.complex(torch.tensor([0], dtype=torch.float32), torch.tensor([1], dtype=torch.float32))*ang)

    term1 = mag_M_complex*exp1
    term2 = mag_M_complex*exp2

    FFtmp = (term1*term2) / (32**2)

    autocorr = torch.fft.ifft2(FFtmp)
    autocorr = autocorr.real
    autocorr = torch.permute(autocorr, (0, 2, 3, 1))

    rv = torch.tensor(dim//2, dtype=torch.int32)
    #autocorr = torch.roll(autocorr, rv, 1)
    autocorr = torch.roll(autocorr, rv, 2)

    return autocorr


class FFTLoss(nn.Module):
    def __init__(self):
        super(FTLoss, self).__init__()
        self.mae_loss = nn.L1Loss()
    
    def forward(self, input, target):
        input_autocorr = autocorr(input)
        target_autocorr = autocorr(target)
        diff = self.mae_loss(input_autocorr, target_autocorr)
        return diff
    

### testing code

In [ ]:
import torch
import torch.nn as nn

input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5)

mae_loss = nn.L1Loss()
output = mae_loss(input, target)
output.backward()

print('input: ', input)
print('target: ', target)
print('output: ', output)